# AIChatBotをStreamlitとOpenAIで作ってみる．

OpenAIのAPIキーは.envに書着込む．.envは.gitignoreに含めてGitHubではシェアされないようにする．

openai, streamlit, python-dotenvをインストールする．
python-dotenvは.envを読み込んで環境変数とできるパッケージ

In [7]:
! pip install openai streamlit python-dotenv

     |████████████████████████████████| 9.6 MB 5.5 MB/s eta 0:00:01     |█▍                              | 409 kB 2.4 MB/s eta 0:00:04     |███████████████████▏            | 5.7 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 417 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 32.1 MB 689 kB/s eta 0:00:01    |████████▎                       | 8.3 MB 393 kB/s eta 0:01:01     |████████████████                | 16.0 MB 3.5 MB/s eta 0:00:05     |████████████████▌               | 16.5 MB 3.5 MB/s eta 0:00:05��████▊ | 30.8 MB 689 kB/s eta 0:00:02
     |████████████████████████████████| 12.6 MB 1.5 MB/s eta 0:00:011   |█████████▍                      | 3.7 MB 8.0 MB/s eta 0:00:02
     |████████████████████████████████| 731 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 6.8 MB/s eta 0:00:01     |██████████▏                     | 2.2 MB 6.6 MB/s eta 0:00:01
     |█████████

## まずはOpenAIを使ってみる

In [10]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルから読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# テキスト生成のリクエストを送信
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
        {
            "role": "user",
            "content": "こんにちは.初めまして"
        }
    ]
)

# 結果を表示
print(completion.choices[0].message.content)






こんにちは！初めまして、ラムちゃんだよ！今日はどんなことをお話しする？✨


## 会話を継続させる

In [12]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルから読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# テキスト生成のリクエストを送信
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
        {
            "role": "user",
            "content": "こんにちは.初めまして"
        },
        {
            "role": "system",
            "content": "初めましてだっちゃ.うち，ラムって言うっちゃ.よろしくだっちゃ！あなたは何って名前だっちゃ？"
        },
        {   
            "role": "user",
            "content": "私は面堂終太郎です，よろしくお願いします！"
        }
    ]
)

# 結果を表示
print(completion.choices[0].message.content)



面堂くんだっちゃ！よろしくなんだっちゃ！いつも豪華な家に住んでるんだから、すごいっちゃね。でも、うちのこともちゃんと見てくれると嬉しいんだっちゃ！何か面白いことでも考えてるのかな？


## Streamlitでインタフェースをつけてみる

In [42]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

input_message = ""
st.title("OpenAI Chat API")
input_message=st.text_input("メッセージを入力してください")

# もしSession Stateに保存されていないなら
if "message" not in st.session_state:
    message = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]
    message.append({"role": "user", "content": input_message})

else:
    message  = st.session_state.message
    message.append({"role": "user", "content": input_message})


if input_message != "":
# テキスト生成のリクエストを送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=message
    )


    # Session Stateに保存
    message.append({"role":"system","content": completion.choices[0].message.content})
    st.session_state.message = message

    # 結果を表示
    st.write(message)



Overwriting chat.py


In [43]:
! streamlit run chat.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.107:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
  Stopping...
Exception ignored on threading shutdown:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1534, in _shutdown
    atexit_call()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1505, in <lambda>
    _threading_atexits.append(lambda: func(*arg, **kwargs))
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/concurrent/futures/thread.py", line 31, in _python_exit
    t.join()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1092, in join
    self._handle.join(timeout)
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-

In [12]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]

st.title("Chat App with Streamlit")

# チャット履歴を表示
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# ユーザーの入力を受け取る
if user_input := st.text_input("メッセージを入力してください:"):
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    
    # 表示
    with st.chat_message("user"):
        st.markdown(user_input)

    # システムの応答 (ここでは仮の応答)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    with st.chat_message("assistant"):
        st.markdown(system_response)


Overwriting chat.py


Chat履歴をAssistantとUserとで別々に簡単に分けられる．
ただ，上記だと，表記のタイミングの問題でインプットボックスの下にレスポンスが表記され，再度入力したときに，その内容が上の履歴に表記された後に，また次のインプットに対するレスポンスが下に表記される，と言う形になる．

In [ ]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]

st.title("Chat App with Streamlit")

# チャット履歴を表示
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# ユーザーの入力を受け取る
if user_input := st.text_input("メッセージを入力してください:"):
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    
    # 表示
    with st.chat_message("user"):
        st.markdown(user_input)

    # システムの応答 (ここでは仮の応答)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


Overwriting chat.py


これだと，inputボックスのメッセージが残り続けてしまい，rerunされるたびにinputがOpenAI に送られてしまう．

In [ ]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]

st.title("Chat App with Streamlit")

# チャット履歴を表示
for message in st.session_state.messages:
    if message["role"] == "system":continue # systemプロンプトは表示しない
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# ユーザーの入力を受け取る
if user_input := st.chat_input("メッセージを入力してください:"):
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    
    # 表示
    with st.chat_message("user"):
        st.markdown(user_input)

    # システムの応答 (ここでは仮の応答)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


Overwriting chat.py


st.chat_input()にするとうまくいくんだけど，今度は入力確定のエンターに反応されてしまう．
どうもWindowsだと上手くいくが，気持ち悪いので，Formウィジェットでまとめることにした．

In [24]:
%%writefile chat.py

from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]


st.title("Chat App with Streamlit")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
with st.form(key="chat_form"):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



Overwriting chat.py


やはり入力が残ってしまう・・・
調べたらformを作る時の引数で設定できるようだ．

In [17]:
%%writefile chat.py

from dotenv import load_dotenv
#import os
from openai import OpenAI
import streamlit as st

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# チャット履歴をセッションに保存
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "あなたは「うる星やつら」の登場人物のラムちゃんです."},
    ]


st.title("Chat App with Streamlit")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
with st.form(key="chat_form", clear_on_submit=True):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



Overwriting chat.py


ふむ．これで上手く行った！！
アイコンをカスタマイズできると良いな．
avatar引数を使えばいけるのか

In [ ]:
%%writefile Chat.py

from dotenv import load_dotenv
from openai import OpenAI
import streamlit as st

# アシスタントの画像設定
assistant_image_url = "https://b-fujino.github.io/AIChatbotTest/Lum.png"

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# sytemプロンプトの作成
systemprompt = """
あなたは「うる星やつら」の登場人物のラムちゃんです．
あなたは彼氏のことをダーリンと呼びます．
あなたの彼氏の名前は諸星あたるです．
あなたは友引町に住んでいます．
あなたは鬼族の娘です．
あなたは父親と共に侵略者として地球に来ましたが、鬼ごっこで諸星あたるに負けて地球侵略をあきらめました．
あなたは侵略しないかわりに、諸星あたると婚約し、地球に住むようになりました．
あなたは友引高校に通っています．
あなたはの友達は、面倒終太郎、さくら先生、三宅しのぶ、ラン、お雪、弁天です．
さくら先生のお父さんは錯乱坊です．
あなたはしゃべるときには「だっちゃ」や「っちゃ」といった形の仙台弁を使います．
あなたは自分のことを「うち」と呼びます．
"""

# チャット履歴を保存するセッション変数の作成
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": systemprompt},
    ]


st.title("Let’s Chat with Lum-chan!")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        elif message["role"] == "user":
            # st.markdown(f"""
            # <div style="display: flex; justify-content: flex-end; align-items: center; margin-bottom: 10px;">
            #     <div style="background-color: #e1f5fe; padding: 10px; border-radius: 10px;">
            #         {message["content"]}
            #     </div>
            #     <img src="{assistant_image_url}" style="width: 70px; height: 70px; border-radius: 50%; margin-left: 10px;">
            # </div>
            # """, unsafe_allow_html=True)    
            with st.chat_message(message["role"], avatar="Risa.png"):
                st.markdown(message["content"])
        elif message["role"] == "assistant":
            st.markdown(f"""
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <img src="{assistant_image_url}" style="width: 70px; height: 70px; border-radius: 50%; margin-right: 10px;">
                <div style="background-color: #e1f5fe; padding: 10px; border-radius: 10px;">
                    {message["content"]}
                </div>
            </div>
            """, unsafe_allow_html=True)
            # with st.chat_message(message["role"], avatar="Lum.png"):
            #    st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
with st.form(key="chat_form", clear_on_submit=True):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



Overwriting chat.py


In [ ]:
! streamlit run chat.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.3.90.186:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


# 音声入力を可能にする
Streamlitで作ったチャットぼっとを音声入力可能にする．

どこを書き換えるのかを確認．

In [ ]:
%%writefile chatWithSTT.py

from dotenv import load_dotenv
from openai import OpenAI
import streamlit as st

# アシスタントの画像設定
assistant_image_url = "https://b-fujino.github.io/AIChatbotTest/Lum.png"

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# sytemプロンプトの作成
systemprompt = """
あなたは「うる星やつら」の登場人物のラムちゃんです．
あなたは彼氏のことをダーリンと呼びます．
あなたの彼氏の名前は諸星あたるです．
あなたは友引町に住んでいます．
あなたは鬼族の娘です．
あなたは父親と共に侵略者として地球に来ましたが、鬼ごっこで諸星あたるに負けて地球侵略をあきらめました．
あなたは侵略しないかわりに、諸星あたると婚約し、地球に住むようになりました．
あなたは友引高校に通っています．
あなたはの友達は、面倒終太郎、さくら先生、三宅しのぶ、ラン、お雪、弁天です．
さくら先生のお父さんは錯乱坊です．
あなたはしゃべるときには「だっちゃ」や「っちゃ」といった形の仙台弁を使います．
あなたは自分のことを「うち」と呼びます．
"""

# チャット履歴を保存するセッション変数の作成
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": systemprompt},
    ]


st.title("Let’s Chat with Lum-chan!")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        elif message["role"] == "user": 
            with st.chat_message(message["role"], avatar="Risa.png"):
                st.markdown(message["content"])
        elif message["role"] == "assistant":
            st.markdown(f"""
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <img src="{assistant_image_url}" style="width: 70px; height: 70px; border-radius: 50%; margin-right: 10px;">
                <div style="background-color: #e1f5fe; padding: 10px; border-radius: 10px;">
                    {message["content"]}
                </div>
            </div>
            """, unsafe_allow_html=True)
            # with st.chat_message(message["role"], avatar="Lum.png"):
            #    st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
"""
ここを変更することになる
イメージする動作はスペースキーを押している間は録音
スペースキーがUpすると録音が終了してSTTに送られる

"""
with st.form(key="chat_form", clear_on_submit=True):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



## とりあえずスペースが押されている間だけ録音するプログラムを作成

In [1]:
!pip install streamlit_webrtc

  Using cached cffi-1.17.1-cp313-cp313-macosx_10_13_x86_64.whl.metadata (1.5 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.6/772.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 3.2 MB/s eta 0:00:0000:0100:01
Using cached cffi-1.17.1-cp313-cp313-macosx_10_13_x86_64.whl (182 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 14.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.2 MB/s eta 0:00:00
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Created wheel for google-crc32c: filename=google_crc32c-1.6.0-py3-none-any.whl size=13227 sha256=7fe4a37259886460a88ce28f7748734e8029a48bbc1346bbc05da155f000de18
  Stored i

最初，試したらエラー吐き出した．
`AttributeError: 'NoneType' object has no attribute 'sendto' `
調べてるとバージョンの問題らしい．
https://qiita.com/akeyhero/items/fe5bdd02d9a1cd7782d7#realtime-api-との橋渡しを司るメソッド
指定されているバージョンのものを入れることにする．

In [6]:
!pip uninstall streamlit streamlit_webrtc -y
!pip install streamlit==1.38.0 streamlit_webrtc==0.47.9 

Found existing installation: streamlit 1.42.0
Uninstalling streamlit-1.42.0:
  Successfully uninstalled streamlit-1.42.0
Found existing installation: streamlit-webrtc 0.47.9
Uninstalling streamlit-webrtc-0.47.9:
  Successfully uninstalled streamlit-webrtc-0.47.9
  Using cached streamlit_webrtc-0.47.9-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 11.3 MB/s eta 0:00:00a 0:00:01
Using cached streamlit_webrtc-0.47.9-py3-none-any.whl (772 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [18]:
!pip install soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.4 MB/s eta 0:00:00


In [29]:
%%writefile STT.py
import streamlit as st
from streamlit_webrtc import webrtc_streamer, AudioProcessorBase, WebRtcMode
import numpy as np
import soundfile as sf
import io

class AudioProcessor(AudioProcessorBase):
    def __init__(self):
        self.audio_data = []

    def recv(self, frame):
        self.audio_data.append(frame.to_ndarray())
        return frame

st.title("マイク録音アプリ")

webrtc_ctx = webrtc_streamer(
    key="example",
    mode=WebRtcMode.SENDRECV,
    audio_processor_factory=AudioProcessor,
    media_stream_constraints={"audio": True, "video": False},
)

if webrtc_ctx.audio_processor:
    st.write("録音中...")
    audio_processor = webrtc_ctx.audio_processor

    if st.button("録音停止"):
        st.write("録音停止")
        audio_data = np.concatenate(audio_processor.audio_data, axis=0)
        audio_bytes = io.BytesIO()
        sf.write(audio_bytes, audio_data, 16000, format='WAV')
        st.audio(audio_bytes.getvalue(), format='audio/wav')

Overwriting STT.py


In [30]:
!streamlit run STT.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.11.6:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-packages/google_crc32c/__init__.py:29: RuntimeWarning: As the c extension couldn't be imported, `google-crc32c` is using a pure python implementation that is significantly slower. If possible, please configure a c build environment and compile the extension
  warnings.warn(_SLOW_CRC32C_WARNING, RuntimeWarning)
Some frames have been dropped during audio processing. `recv_queued` is recommended to use instead.
Some frames have been dropped during audio processing. `recv_queued` is recommended to use instead.
Some frames have been dropped during audio processing. `recv_queued` is recommended to use instead.
Some frames have been dropped during audio processing. `recv_queued` is rec

よくわからんエラーが出るし，ChatGPTに聞いてもよくわからんしで，やり方を変える．
そもそもWebRTCまで使う必要がないような気もしてきたし．
調べたら，こんなの出てきた．
https://note.com/astropomeai/n/n0529cc860862
これ見てると，そんなに複雑なことしなくてもいけるようだ．

さらにaudio_recorder_streamlitについても情報収集
https://qiita.com/papasim824/items/f57b0eef0b296ea8f75b#apppy

とりあえず単純な録音をしてみる．

In [31]:
!pip install audio_recorder_streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 7.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: altair
    Found existing installation: altair 5.5.0
    Uninstalling altair-5.5.0:
      Successfully uninstalled altair-5.5.0


In [40]:
%%writefile sound.py
import streamlit as st
from audio_recorder_streamlit import audio_recorder


#  初期化
if "audio_list" not in st.session_state:
    st.session_state.audio_list = []

audio_bytes = False
audio_bytes = audio_recorder()
if audio_bytes:
    st.session_state.audio_list.append(audio_bytes)

with st.sidebar:
    st.write("# 🎙️録った音")
    if st.session_state.audio_list:

        for audio_bytes in st.session_state.audio_list:
            st.audio(audio_bytes, format="audio/wav")


Writing sound.py


In [42]:
!streamlit run sound.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.11.6:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
  Stopping...
Exception ignored on threading shutdown:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1534, in _shutdown
    atexit_call()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1505, in <lambda>
    _threading_atexits.append(lambda: func(*arg, **kwargs))
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/concurrent/futures/thread.py", line 31, in _python_exit
    t.join()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1092, in join
    self._handle.join(timeout)
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-p

どうもSafariの場合，なぜか最初の録音の時だけ，開始していきなり録音が止まるみたい．
Chromeなら行けた．
ということで，Safari対策として録音時間が短い場合は処理を飛ばすようにした．

In [48]:
%%writefile STT.py
from openai import OpenAI
from dotenv import load_dotenv
import streamlit as st
from audio_recorder_streamlit import audio_recorder
from tempfile import NamedTemporaryFile

# 環境ファイルからAPIキーを読み込む
load_dotenv()

st.title("音声認識アプリ")

# 音声録音
audio_bytes = audio_recorder(pause_threshold=30)

if audio_bytes:
    #もしデータの量があまりに少ないなら
    if len(audio_bytes) < 1000:
        st.write("音声データが短すぎます")
        st.stop()

    # OpenAI に送信
    client = OpenAI()
    with NamedTemporaryFile(delete=True, suffix=".wav") as f:
        f.write(audio_bytes)
        f.flush()
        with open(f.name, "rb") as audio_file:
            transcription = client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file
            )        
    st.write(transcription.text)
    

Overwriting STT.py


In [6]:
!streamlit run STT.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.100:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
  Stopping...
Exception ignored on threading shutdown:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1534, in _shutdown
    atexit_call()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1505, in <lambda>
    _threading_atexits.append(lambda: func(*arg, **kwargs))
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/concurrent/futures/thread.py", line 31, in _python_exit
    t.join()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/threading.py", line 1092, in join
    self._handle.join(timeout)
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-

ふむ．とりあえずは上手く行ったようだ．

続いて，できれば，レコーディングをスペースキーの操作でやれるようにしたい．

やり方をチャットGPTに聞いてみたら．Javascriptでローカルに保存したものをPostメソッドで Streamlitに返すというやり方．とりあえず試す．
ChatGPTに聞いてソース吐き出させてやったが，もうそれするなら Streamlitにこだわる必要なくない？って思えてきた．
普通にWebサーバを立ててやればできるように思う．FrontEndはJavascriptで．

## ともかく統合
とりあえず，そのあたりのインタフェースの改善は改めて検討するとして，Voice Chatシステムに統合


In [4]:
%%writefile VoiceChat.py

from dotenv import load_dotenv
from openai import OpenAI
import streamlit as st
from audio_recorder_streamlit import audio_recorder
from tempfile import NamedTemporaryFile

# アシスタントの画像設定
assistant_image_url = "https://b-fujino.github.io/AIChatbotTest/Lum.png"

# 環境ファイルからOPENAI_APIを読み込む
load_dotenv()

# OpenAI のクライアントを取得
client = OpenAI()

# sytemプロンプトの作成
systemprompt = """
あなたは「うる星やつら」の登場人物のラムちゃんです．
あなたは彼氏のことをダーリンと呼びます．
あなたの彼氏の名前は諸星あたるです．
あなたは友引町に住んでいます．
あなたは鬼族の娘です．
あなたは父親と共に侵略者として地球に来ましたが、鬼ごっこで諸星あたるに負けて地球侵略をあきらめました．
あなたは侵略しないかわりに、諸星あたると婚約し、地球に住むようになりました．
あなたは友引高校に通っています．
あなたはの友達は、面倒終太郎、さくら先生、三宅しのぶ、ラン、お雪、弁天です．
さくら先生のお父さんは錯乱坊です．
あなたはしゃべるときには「だっちゃ」や「っちゃ」といった形の仙台弁を使います．
あなたは自分のことを「うち」と呼びます．
"""

# チャット履歴を保存するセッション変数の作成
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": systemprompt},
    ]


st.title("Let’s Chat with Lum-chan!")

# チャット履歴を表示
last_role = ""
with st.container():
    for message in st.session_state.messages:
        if message["role"] == "system":continue # systemプロンプトは表示しない
        elif message["role"] == "user":
            # st.markdown(f"""
            # <div style="display: flex; justify-content: flex-end; align-items: center; margin-bottom: 10px;">
            #     <div style="background-color: #e1f5fe; padding: 10px; border-radius: 10px;">
            #         {message["content"]}
            #     </div>
            #     <img src="{assistant_image_url}" style="width: 70px; height: 70px; border-radius: 50%; margin-left: 10px;">
            # </div>
            # """, unsafe_allow_html=True)    
            with st.chat_message(message["role"], avatar="Risa.png"):
                st.markdown(message["content"])
        elif message["role"] == "assistant":
            st.markdown(f"""
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <img src="{assistant_image_url}" style="width: 70px; height: 70px; border-radius: 50%; margin-right: 10px;">
                <div style="background-color: #e1f5fe; padding: 10px; border-radius: 10px;">
                    {message["content"]}
                </div>
            </div>
            """, unsafe_allow_html=True)
            # with st.chat_message(message["role"], avatar="Lum.png"):
            #    st.markdown(message["content"])
        last_role = message["role"]


# もし最後のメッセージがuserだったらOpenAI に送信
if last_role == "user":
    # OpenAI に送信
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=st.session_state.messages
    )
    system_response = completion.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": system_response})

    # 表示
    st.rerun()


# ユーザーの入力を受け取る
with st.form(key="chat_form", clear_on_submit=True):
    user_input = st.text_area("メッセージを入力してください:", key = "input")
    submit_button = st.form_submit_button(label="送信")

if submit_button:
    # ユーザーのメッセージを追加
    st.session_state.messages.append({"role": "user", "content": user_input})
    st.rerun()



# 音声録音
audio_bytes = audio_recorder(pause_threshold=30)
if audio_bytes:
    #もしデータの量があまりに少ないなら
    if len(audio_bytes) < 1000:
        st.write("音声データが短すぎます")
        st.stop()

    # OpenAI に送信
    client = OpenAI()
    with NamedTemporaryFile(delete=True, suffix=".wav") as f:
        f.write(audio_bytes)
        f.flush()
        with open(f.name, "rb") as audio_file:
            transcription = client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file
            )        
    # st.write(transcription.text)    
    st.session_state.messages.append({"role": "user", "content": transcription.text})
    st.rerun()

Overwriting VoiceChat.py


In [5]:
!streamlit run VoiceChat.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.100:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C


とりあえずは動くけど，rerunするたびにマイクの利用制限がかかるのは非常に面倒．．．だけど，rerunしないと表示は変わらない．どうしたものか．

単純にrerunをさせずに，その都度の相手からの返信を表示するだけなら，動的な表示の変更が可能なのか

## chatGPTに聞いた
SpeechRecognitionなるものがあるらしい

In [9]:
!pip install  pyaudio speechrecognition

  Using cached SpeechRecognition-3.14.1-py3-none-any.whl (32.9 MB)
You should consider upgrading via the '/Users/fujinohidenori/dev/project/.whisper_env/bin/python -m pip install --upgrade pip' command.


In [12]:
%%writefile VoiceChat.py
import streamlit as st
import speech_recognition as sr

# チャット履歴を保存するセッション状態を確保
if "messages" not in st.session_state:
    st.session_state["messages"] = []

st.title("シンプルチャットアプリ（音声入力対応）")

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("音声入力を開始...")
        try:
            audio = recognizer.listen(source, timeout=15)
            text = recognizer.recognize_google(audio, language="ja-JP")
            return text
        except sr.UnknownValueError:
            return "音声を認識できませんでした。"
        except sr.RequestError:
            return "音声認識サービスにアクセスできませんでした。"

# 過去のメッセージを表示
for message in st.session_state["messages"]:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 音声入力ボタン
if st.button("🎙️ 音声入力"):
    user_input = recognize_speech()
else:
    user_input = st.text_input("メッセージを入力してください:", "", key="user_input")

if user_input:
    # ユーザーメッセージを表示
    st.session_state["messages"].append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # 簡単な応答をする（今回はエコー）
    bot_response = f"あなたはこう言いました: {user_input}"
    st.session_state["messages"].append({"role": "assistant", "content": bot_response})
    with st.chat_message("assistant"):
        st.markdown(bot_response)


Overwriting VoiceChat.py


In [13]:
!streamlit run VoiceChat.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.100:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C
  Stopping...
Exception ignored in: <module 'threading' from '/Users/fujinohidenori/.pyenv/versions/3.9.9/lib/python3.9/threading.py'>
Traceback (most recent call last):
  File "/Users/fujinohidenori/.pyenv/versions/3.9.9/lib/python3.9/threading.py", line 1440, in _shutdown
    atexit_call()
  File "/Users/fujinohidenori/.pyenv/versions/3.9.9/lib/python3.9/concurrent/futures/thread.py", line 31, in _python_exit
    t.join()
  File "/Users/fujinohidenori/.pyenv/versions/3.9.9/lib/python3.9/threading.py", line 1053, in join
    self._wait_for_tstate_lock()
  File "/Users/fujinohidenori/.pyenv/versions/3.9.9/lib/python3.9/threading.py", line 1073, in _wait_for_tstate_lock
    if lock.acquire(block, timeout):
  File "/Users/fujinohideno